In [4]:
import h5py
import os
import json
import matplotlib.pyplot as plt

import os
import sys
sys.path.append("./../")
import numpy as np
import matplotlib.pyplot as plt
import pdb
import pandas as pd
import yaml
import h5py
#import vicon_imu_data_process.process_landing_data as pro_rd
#import assessments.scores as es_as
import pickle
import seaborn as sns
import copy
import re
import json

#from vicon_imu_data_process.const import FEATURES_FIELDS, LABELS_FIELDS, DATA_PATH
##from vicon_imu_data_process.const import DROPLANDING_PERIOD, RESULTS_PATH
#from vicon_imu_data_process import const
#import vicon_imu_data_process.augmentation_data as aug

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
import time as localtimepkg

#from estimation_study import *
#from estimation_assessment.visualization import *
#import pdb
import re
import logging
logging.basicConfig(level=logging.ERROR)
import sys
sys.path.append("./../")


import pandas as pd
data_dir = "./../data/"

datafile_name="17_23年区域经济数据.xls"

data_path=os.path.join(data_dir, datafile_name)
region_ecodata_dict = pd.read_excel(data_path, index_col=0,header=0,sheet_name=None,skiprows=0)
list_region_ecodata=[]
for key, value in region_ecodata_dict.items():
    tmp_value=value
    tmp_value["年"]=key
    list_region_ecodata.append(tmp_value)

all_years_region_ecodata=pd.concat(list_region_ecodata)
all_years_region_ecodata.drop(labels=["中国"],axis=0,inplace=True)
all_years_region_ecodata.fillna(value=0.0,inplace=True)
#all_years_region_ecodata.reset_index()
print(all_years_region_ecodata.columns)
print(all_years_region_ecodata.head())




regions=list(set(all_years_region_ecodata["省"].values))
subjects_trials_data = {}
for region in regions:
    data = all_years_region_ecodata[all_years_region_ecodata["省"]==region]
    subjects_trials_data[region]={}
    for idx in range(4):
        trial_years=[str(tmp+idx) for tmp in [2017, 2018, 2019, 2020]]
        tmp_data = data[data["年"].isin(trial_years)]
        tmp_data2 = tmp_data.drop(["省","年"],axis=1)
        subjects_trials_data[region][str(idx)]=tmp_data2

DATA_PATH="./../data/"
def save_subjects_dataset_to_a_h5(subjects_trials_dataset, data_file_name="example_data.hdf5"):
    # declare h5 file
    h5format_dataset=os.path.join(DATA_PATH,data_file_name)
    print("dataset was stored at:", h5format_dataset)
    
    # remove the exist h5 file
    if os.path.exists(h5format_dataset):
        try:
            os.remove(h5format_dataset)
        except IOError:
            print("cannot remove h5 file")
            
    # save the h5 file
    with h5py.File(h5format_dataset, "w") as f:
        subjects_ids_names  = list(subjects_trials_dataset.keys())
        f.attrs['subjects'] = subjects_ids_names
        for subject_id_name in subjects_ids_names: # subjects
            sub=f.create_group(subject_id_name)
            try:
                for trial, a_trial_data in subjects_trials_dataset[subject_id_name].items(): # trial numbers
                    sub.create_dataset(trial,data=a_trial_data)
                sub.attrs['columns'] = list(a_trial_data.columns)
            except Exception as e: 
                print(e)
                print(termcolor.colored("subject: {} h5 file path in trial: {} is wrong".format(subject_id_name,trial),'red'))
                pdb.set_trace()

save_subjects_dataset_to_a_h5(subjects_trials_data, "ecodata.hdf5")

Index(['省', '城投债余额(亿)', '城投平台数量(家)', '近一月发行量(亿)', '近一月净融资亿)', 'GDP(亿)',
       '人均GDP(元)', 'GDP增速(%)', '一般公共预算收入(亿)', '转移性收入(亿)', '上级补助收入(亿)',
       '转移支付收入(亿)', ' 税收收入(亿)', '政府性基金收入(亿)', '土地出让收入(亿)', '财政自给率(%)',
       ' 地方政府债务余额(亿元)', '负债率(%)', '债务率(%)', '债务率(宽口径,本级,%)', '债务率(宽口径,全辖,%)',
       '城投有息债务合计(亿)', '城投有息债务合计/地区综合财力(%)', '地方综合财力(亿元)', '年'],
      dtype='object')
        省  城投债余额(亿)  城投平台数量(家)  近一月发行量(亿)  近一月净融资亿)    GDP(亿)  人均GDP(元)  \
区域名称                                                                      
重庆市   重庆市   7354.13        154      85.07     32.37  19500.27   63689.0   
上海市   上海市   4006.82         85      19.50    -45.00  30632.99  126634.0   
北京市   北京市   6482.66         75     134.50     73.50  28014.94  128994.0   
天津市   天津市   3839.56         61      42.00      7.00  18549.19  118944.0   
江苏省   江苏省  32095.42        723     472.40     91.70  85900.94  107189.0   

      GDP增速(%)  一般公共预算收入(亿)  转移性收入(亿)  ...  财政自给率(%)   地方政府债务余额(亿元)  负债率(%)  \
区域名称             